# Install Packages on Google Colab and import Packages

In [1]:
!pip install vtk
!pip install pyevtk
import torch
import numpy as np
from matplotlib import pyplot as plt
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pdb
import csv
from torch.utils.data import DataLoader, TensorDataset,RandomSampler
from math import exp, sqrt,pi
import time
import vtk
from vtk.util import numpy_support as VN
import math
import os.path
from openpyxl import load_workbook
from openpyxl.workbook import Workbook
from pyevtk.hl import pointsToVTK

     |████████████████████████████████| 88.3 MB 48 kB/s 
     |████████████████████████████████| 1.1 MB 40.1 MB/s 
     |████████████████████████████████| 271 kB 48.4 MB/s 
     |████████████████████████████████| 160 kB 44.9 MB/s 
     |████████████████████████████████| 192 kB 43.1 MB/s 


# Connect to google drive to load the files and write the outputs

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# The Principal function that contains Siren-Based Network, training, and output writing

In [3]:
# Two Helper function
def exists(val):
    return val is not None

def cast_tuple(val, repeat = 1):
    return val if isinstance(val, tuple) else ((val,) * repeat)


In [4]:
# choose that you want to run the program on CPU or GPU
Flag_GPU = 1
if (Flag_GPU==1):
	device = torch.device("cuda") 
else:
	device = torch.device("cpu")

def geo_train(device,x_in,y_in,xb,yb,ub,vb,xd,yd,ud,vd,batchsize,learning_rate,epochs,path,Flag_batch,Diff,rho,Flag_BC_exact,Lambda_BC,nPt,T,xb_inlet,yb_inlet,ub_inlet,vb_inlet ):
	x_asli= x_in
	y_asli= y_in
	if (Flag_batch):
	 x = torch.Tensor(x_in).to(device)
	 y = torch.Tensor(y_in).to(device)
	 xb = torch.Tensor(xb).to(device)
	 yb = torch.Tensor(yb).to(device)
	 ub = torch.Tensor(ub).to(device)
	 vb = torch.Tensor(vb).to(device)
	 xd = torch.Tensor(xd).to(device)
	 yd = torch.Tensor(yd).to(device)
	 ud = torch.Tensor(ud).to(device)
	 vd = torch.Tensor(vd).to(device)
	 xb_inlet = torch.Tensor(xb_inlet).to(device)
	 yb_inlet = torch.Tensor(yb_inlet).to(device)
	 ub_inlet = torch.Tensor(ub_inlet).to(device)
	 vb_inlet = torch.Tensor(vb_inlet).to(device)
	 if(Flag_GPU==1): #Cuda slower in double?   For Float this means that it only has four decimal places while Double still has twelve
					# Therefore, if we change the double to fload the usage of the memory will be lower, so network will be trained with a better pace using GPU
		 x = x.type(torch.cuda.FloatTensor)
		 y = y.type(torch.cuda.FloatTensor)
		 xb = xb.type(torch.cuda.FloatTensor)
		 yb = yb.type(torch.cuda.FloatTensor)
		 ub = ub.type(torch.cuda.FloatTensor)
		 vb = vb.type(torch.cuda.FloatTensor)
		 xb_inlet = xb_inlet.type(torch.cuda.FloatTensor)
		 yb_inlet = yb_inlet.type(torch.cuda.FloatTensor)
		 ub_inlet = ub_inlet.type(torch.cuda.FloatTensor)
		 vb_inlet = vb_inlet.type(torch.cuda.FloatTensor)
		 xd = xd.type(torch.cuda.FloatTensor)
		 yd = yd.type(torch.cuda.FloatTensor)
		 ud = ud.type(torch.cuda.FloatTensor)
		 vd = vd.type(torch.cuda.FloatTensor)

	 # generate the input dataset (it was 2D)
	 dataset = TensorDataset(x,y)
	 dataloader = DataLoader(dataset, batch_size=batchsize,shuffle=True,num_workers = 0, drop_last = True )

	else:
	 x = torch.Tensor(x_in).to(device)
	 y = torch.Tensor(y_in).to(device) 

	h_n = 128					 #Width for u,v,p # number of hidden neurons
	input_n = 2 			 # this is what our answer is a function of (x,y)

	class Sine(nn.Module):
		def __init__(self, w0 = 1.):
			super().__init__()
			self.w0 = w0
		def forward(self, x):
			return torch.sin(self.w0 * x)

	class Siren(nn.Module):
		def __init__(self, dim_in, dim_out, w0 = 1., c = 6., is_first = False, use_bias = True, activation = None):
			super().__init__()
			self.dim_in = dim_in
			self.is_first = is_first
			if (Flag_GPU==1):
				weight = torch.zeros(dim_out, dim_in , device="cuda")
			else:
				weight = torch.zeros(dim_out, dim_in)
			
			if (Flag_GPU==1):
				bias = torch.zeros(dim_out, device="cuda") if use_bias else None
			else:
				bias = torch.zeros(dim_out) if use_bias else None

			self.init_(weight, bias, c = c, w0 = w0)

			self.weight = nn.Parameter(weight)
			self.bias = nn.Parameter(bias) if use_bias else None
			self.activation = Sine(w0) if activation is None else activation

		def init_(self, weight, bias, c, w0):
			dim = self.dim_in

			w_std = (1 / dim) if self.is_first else (math.sqrt(c / dim) / w0)
			weight.uniform_(-w_std, w_std)


			if exists(bias):
				bias.uniform_(-w_std, w_std)

		def forward(self, x):
			out =  F.linear(x, self.weight, self.bias)
			out = self.activation(out)
			return out

	class SirenNet(nn.Module):
		def __init__(self, dim_in, dim_hidden, dim_out, num_layers, w0 = 1., w0_initial = 30., use_bias = True, final_activation = None):
			super().__init__()
			self.num_layers = num_layers
			self.dim_hidden = dim_hidden

			self.layers = nn.ModuleList([])
			for ind in range(num_layers):
				is_first = ind == 0
				layer_w0 = w0_initial if is_first else w0
				layer_dim_in = dim_in if is_first else dim_hidden

				self.layers.append(Siren(
						dim_in = layer_dim_in,
						dim_out = dim_hidden,
						w0 = layer_w0,
						use_bias = use_bias,
						is_first = is_first
				))

			final_activation = nn.Identity() if not exists(final_activation) else final_activation
			self.last_layer = Siren(dim_in = dim_hidden, dim_out = dim_out, w0 = w0, use_bias = use_bias, activation = final_activation)


		def forward(self, x, mods = None):
			x=x*2 -1
			mods = cast_tuple(mods, self.num_layers)
	 
			for layer, mod in zip(self.layers, mods):
				x = layer(x)

				if exists(mod):
					x *= rearrange(mod, 'd -> () d')

			return self.last_layer(x)


	net2 = net = SirenNet(
		dim_in = 2,                        # input dimension, ex. 2d coor
		dim_hidden = 128,                  # hidden dimension
		dim_out = 3,                       # output dimension, ex. rgb value
		num_layers = 4,                    # number of layers
		w0_initial = 15.                   # different signals may require different omega_0 in the first layer - this is a hyperparameter
)
  

### Optimizers of Networks
	optimizer_Siren = optim.Adam(net2.parameters(), lr=learning_rate, betas = (0.9,0.99),eps = 10**-15)
	

###### Define Losses - We have 3 losses containing: Loss of the equations, loss of the sensor Data, Loss of BC (no slip in wall boundries)
	def criterion(x,y):

		x.requires_grad = True
		y.requires_grad = True
		net_in = torch.cat((x,y),1)
		outnet = net2(net_in)
		u= outnet[:,0]
		v= outnet[:,1]
		P= outnet[:,2]
		u = u.view(len(u),-1)
		v = v.view(len(v),-1)
		P = P.view(len(P),-1)
		# prepare the gradient to make the equations as losses
		u_x = torch.autograd.grad(u,x,grad_outputs=torch.ones_like(x),create_graph = True,only_inputs=True)[0]
		u_xx = torch.autograd.grad(u_x,x,grad_outputs=torch.ones_like(x),create_graph = True,only_inputs=True)[0]
		u_y = torch.autograd.grad(u,y,grad_outputs=torch.ones_like(y),create_graph = True,only_inputs=True)[0]
		u_yy = torch.autograd.grad(u_y,y,grad_outputs=torch.ones_like(y),create_graph = True,only_inputs=True)[0]
		v_x = torch.autograd.grad(v,x,grad_outputs=torch.ones_like(x),create_graph = True,only_inputs=True)[0]
		v_xx = torch.autograd.grad(v_x,x,grad_outputs=torch.ones_like(x),create_graph = True,only_inputs=True)[0]
		v_y = torch.autograd.grad(v,y,grad_outputs=torch.ones_like(y),create_graph = True,only_inputs=True)[0]
		v_yy = torch.autograd.grad(v_y,y,grad_outputs=torch.ones_like(y),create_graph = True,only_inputs=True)[0]
		P_x = torch.autograd.grad(P,x,grad_outputs=torch.ones_like(x),create_graph = True,only_inputs=True)[0]
		P_y = torch.autograd.grad(P,y,grad_outputs=torch.ones_like(y),create_graph = True,only_inputs=True)[0]
		# scale to magnitude the losses ( not necessary )		
		XX_scale = U_scale * (X_scale**2)
		YY_scale = U_scale * (Y_scale**2)
		UU_scale  = U_scale **2
	
		loss_2 = u*u_x / X_scale + v*u_y / Y_scale - Diff*( u_xx/XX_scale  + u_yy /YY_scale  )+ 1/rho* (P_x / (X_scale*UU_scale)   )  #X-dir
		loss_1 = u*v_x / X_scale + v*v_y / Y_scale - Diff*( v_xx/ XX_scale + v_yy / YY_scale )+ 1/rho*(P_y / (Y_scale*UU_scale)   ) #Y-dir
		loss_3 = (u_x / X_scale + v_y / Y_scale) #continuity

		# MSE LOSS
		loss_f = nn.MSELoss()

		#Note our target is zero. It is residual so we use zeros_like
		loss = loss_f(loss_1,torch.zeros_like(loss_1))+  loss_f(loss_2,torch.zeros_like(loss_2))+  loss_f(loss_3,torch.zeros_like(loss_3))

		return loss

	def Loss_BC(xb,yb,ub,vb, xb_inlet, yb_inlet, ub_inlet, x, y ):
		net_in1 = torch.cat((xb, yb), 1)
		outnet = net2(net_in1)
		out1_u = outnet[:,0]
		out1_v = outnet[:,1]
		out1_u = out1_u.view(len(out1_u),-1)
		out1_v = out1_v.view(len(out1_v),-1)

		loss_f = nn.MSELoss()
		loss_noslip = loss_f(out1_u, torch.zeros_like(out1_u)) + loss_f(out1_v, torch.zeros_like(out1_v)) 

		return loss_noslip


	def Loss_data(xd,yd,ud,vd ):
		net_in1 = torch.cat((xd, yd), 1)
		outnet = net2(net_in1)
		out1_u = outnet[:,0]
		out1_v = outnet[:,1]
		out1_u = out1_u.view(len(out1_u),-1)
		out1_v = out1_v.view(len(out1_v),-1)

		loss_f = nn.MSELoss()
		loss_d = loss_f(out1_u, ud) + loss_f(out1_v, vd) 

		return loss_d

	# Main loop

	tic = time.time()
 
	Flag_pretrain = True # True #If true reads the nets from last run
	if(Flag_pretrain & os.path.isfile(path+"sten_data" + ".pt")):
		print('Reading (pretrain) functions first...')
		net2.load_state_dict(torch.load(path+"sten_data" + ".pt"))

		
	if (Flag_schedule):
		scheduler_u = torch.optim.lr_scheduler.StepLR(optimizer_Siren, step_size=step_epoch, gamma=decay_rate)

	if(Flag_batch):# This one uses dataloader
			LOSS = np.zeros(epochs)
			LOSS_eq = np.zeros(epochs)
			LOSS_data = np.zeros(epochs)
			LOSS_BC = np.zeros(epochs)
			zaman = np.zeros(epochs)

			for epoch in range(epochs):

				loss_eqn_tot = 0.
				loss_bc_tot = 0.
				loss_data_tot = 0.
				n = 0
				for batch_idx, (x_in,y_in) in enumerate(dataloader): 
					net2.zero_grad()
					loss_eqn = criterion(x_in,y_in)
					loss_bc = Loss_BC(xb,yb,ub,vb,xb_inlet,yb_inlet,ub_inlet,x,y)
					loss_data = Loss_data(xd,yd,ud,vd)
					loss = loss_eqn + Lambda_BC* loss_bc + Lambda_data*loss_data

					loss.backward()
					optimizer_Siren.step() 
					loss_eqn_tot += loss_eqn
					loss_bc_tot += loss_bc
					loss_data_tot  += loss_data
					n += 1 
					if batch_idx % 40 ==0:
						print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss eqn {:.10f} Loss BC {:.8f} Loss data {:.8f}'.format(
							epoch, batch_idx * len(x_in), len(dataloader.dataset),
							100. * batch_idx / len(dataloader), loss_eqn.item(), loss_bc.item(),loss_data.item()))

				if (Flag_schedule):
						scheduler_u.step()

				loss_eqn_tot = loss_eqn_tot / n
				loss_bc_tot = loss_bc_tot / n
				loss_data_tot = loss_data_tot / n
				loss_avg_tot = loss_eqn_tot + Lambda_BC* loss_bc_tot + Lambda_data*loss_data_tot
				wb = Workbook()
				workbook_name = path+'loss_siren.xlsx'
				wb = load_workbook(workbook_name)
				page = wb.active
				information = [[loss_eqn_tot.cpu().detach().numpy(),loss_bc_tot.cpu().detach().numpy(),loss_data_tot.cpu().detach().numpy(),loss_avg_tot.cpu().detach().numpy()]]
				information=np.array(information)
				for info in information.tolist():
					page.append(info)
				wb.save(filename=workbook_name)
				torch.save(net2.state_dict(),path+"sten_data" + ".pt")

				print('*****Total avg Loss : Loss eqn {:.10f} Loss BC {:.10f} Loss data {:.10f} ****'.format(loss_eqn_tot, loss_bc_tot,loss_data_tot) )
				print('learning rate is ', optimizer_Siren.param_groups[0]['lr'])
		
			if(0): #This causes out of memory in cuda in autodiff
				loss_eqn = criterion(x,y)	
				loss_bc = Loss_BC(xb,yb,ub,vb)
				loss = loss_eqn #+ Lambda_BC* loss_bc
				print('**** Final (all batches) \tLoss: {:.10f} \t Loss BC {:.6f}'.format(
					loss.item(),loss_bc.item()))

	else:
		for epoch in range(epochs):
			##Closure function for LBFGS loop:
			net2.zero_grad()
			loss_eqn = criterion(x,y)
			loss_bc = Loss_BC(xb,yb,cb)
			if (Flag_BC_exact):
				loss = loss_eqn #+ loss_bc
			else:
				loss = loss_eqn + Lambda_BC * loss_bc
			loss.backward()

			optimizer_Siren.step() 

			if epoch % 10 ==0:
				print('Train Epoch: {} \tLoss: {:.10f} \t Loss BC {:.6f}'.format(
					epoch, loss.item(),loss_bc.item()))

	toc = time.time()
	elapseTime = toc - tic
	print ("elapse time in parallel = ", elapseTime)
	###################
	#plot
	if(1):#save network
		torch.save(net2.state_dict(),path+"sten_data" + ".pt")


		print ("Data saved!")


	
	net_in = torch.cat((x.requires_grad_(),y.requires_grad_()),1)
	outneti = net2(net_in)  #evaluate model (runs out of memory for large GPU problems!)
	output_u = outneti[:,0]
	output_v = outneti[:,1]  #evaluate model
	output_p = outneti[:,2]


	output_u = output_u.cpu().data.numpy() #need to convert to cpu before converting to numpy
	output_v = output_v.cpu().data.numpy()
	output_p = output_p.cpu().data.numpy()
	x = x.cpu()
	y = y.cpu()




	plt.figure()
	plt.subplot(2, 1, 1)
	plt.scatter(x.detach().numpy(), y.detach().numpy(), c = output_u , cmap = 'rainbow')
	plt.title('NN results, u')
	plt.colorbar()
	plt.show()
	plt.figure()
	plt.subplot(2, 1, 1)
	plt.scatter(x.detach().numpy(), y.detach().numpy(), c = output_v , cmap = 'rainbow')
	plt.title('NN results, v')
	plt.colorbar()
	plt.show()
 
	npoints = len(x)
	z = np.zeros((npoints,1))
	u = output_u
	v = output_v
	vel_z=np.zeros((npoints,1))
	p = output_p
	v=np.float64(v)
	u=np.float64(u)
	pointsToVTK(path+"./output_siren", x_asli, y_asli, z, data = {"velocity" : (u,v,vel_z), "pressure" : p})
	print('vtu file is constructed')



  
	return net2




# Main code (Input data and preprocessing - define hyper parameteres - run the functions/classes)

## load the mesh file & boundry files. Then, extract and prepare the coordinates of them


In [5]:
# Open the raw Data from google Drive
Directory = "/content/drive/My Drive/stenosis/"   # the address that I've saved my data there, on Google Colab
mesh_file = Directory + "sten_mesh000000.vtu"
bc_file_in = Directory + "inlet_BC.vtk"
bc_file_wall = Directory + "wall_BC.vtk"
File_data = Directory + "velocity_sten_steady.vtu"
fieldname = 'f_5-0' #The velocity field name in the vtk file (see from ParaView)

print ('\nLoading the mesh file and make x & y coordinates ready in variables x & y')

# read the mesh file and find the number of points in it
reader = vtk.vtkXMLUnstructuredGridReader()   # make a reader object
reader.SetFileName(mesh_file)                 # read mesh file using reader object
reader.Update()
data_vtk = reader.GetOutput()
n_points = data_vtk.GetNumberOfPoints()       # find the number of points in our mesh

# define two empty numpy array in order to move the mesh coordinates in them
x_vtk_mesh = np.zeros((n_points,1))
y_vtk_mesh = np.zeros((n_points,1))

VTKpoints = vtk.vtkPoints()

# move the mesh coordinates in x_vtk_mesh & y_vtk_mesh  ( the mesh was 2D )
for i in range(n_points):
	pt_iso  =  data_vtk.GetPoint(i)  # pt_iso has three dim that the first is x-coordinates, y-coordinates, and z-coordinates
	x_vtk_mesh[i] = pt_iso[0]	
	y_vtk_mesh[i] = pt_iso[1]
	VTKpoints.InsertPoint(i, pt_iso[0], pt_iso[1], pt_iso[2])  
point_data = vtk.vtkUnstructuredGrid()
point_data.SetPoints(VTKpoints)

x  = np.reshape(x_vtk_mesh , (np.size(x_vtk_mesh [:]),1))    # move x_vtk_mesh into x 
y  = np.reshape(y_vtk_mesh , (np.size(y_vtk_mesh [:]),1))    # move y_vtk_mesh into y

print ('preparing the mesh coordinates ends and the number of points was:{}'.format(n_points))

print ('\n\nLoading the inlet Boundry file and make the inlet boundry coordinates ready in variables xb_in & yb_in')

# read the inlet boundry file and find the number of points in it
reader = vtk.vtkUnstructuredGridReader()
reader.SetFileName(bc_file_in)
reader.Update()
data_vtk = reader.GetOutput()
n_points = data_vtk.GetNumberOfPoints()

# define two empty numpy array in order to move the inlet boundry coordinates in them
x_vtk_mesh = np.zeros((n_points,1))
y_vtk_mesh = np.zeros((n_points,1))

VTKpoints = vtk.vtkPoints()

# move the inlet boundry coordinates in x_vtk_mesh & y_vtk_mesh
for i in range(n_points):
	pt_iso  =  data_vtk.GetPoint(i)
	x_vtk_mesh[i] = pt_iso[0]	
	y_vtk_mesh[i] = pt_iso[1]
	VTKpoints.InsertPoint(i, pt_iso[0], pt_iso[1], pt_iso[2])
point_data = vtk.vtkUnstructuredGrid()
point_data.SetPoints(VTKpoints)

xb_in  = np.reshape(x_vtk_mesh , (np.size(x_vtk_mesh[:]),1))    # move x_vtk_mesh into xb_in
yb_in  = np.reshape(y_vtk_mesh , (np.size(y_vtk_mesh[:]),1))    # move y_vtk_mesh into yb_in

print ('preparing the inlet boundry coordinates ends and the number of points was:{}'.format(n_points))

print ('\n\nLoading the wall Boundry file and make the wall boundry coordinates ready in variables xb_wall & yb_wall')

# read the wall boundry file and find the number of points in it
reader = vtk.vtkUnstructuredGridReader()
reader.SetFileName(bc_file_wall)
reader.Update()
data_vtk = reader.GetOutput()
n_pointsw = data_vtk.GetNumberOfPoints()

# define two empty numpy array in order to move the wall boundry coordinates in them
x_vtk_mesh = np.zeros((n_pointsw,1))
y_vtk_mesh = np.zeros((n_pointsw,1))
VTKpoints = vtk.vtkPoints()

# move the wall boundry coordinates in x_vtk_mesh & y_vtk_mesh
for i in range(n_pointsw):
	pt_iso  =  data_vtk.GetPoint(i)
	x_vtk_mesh[i] = pt_iso[0]	
	y_vtk_mesh[i] = pt_iso[1]
	VTKpoints.InsertPoint(i, pt_iso[0], pt_iso[1], pt_iso[2])
point_data = vtk.vtkUnstructuredGrid()
point_data.SetPoints(VTKpoints)

xb_wall  = np.reshape(x_vtk_mesh , (np.size(x_vtk_mesh [:]),1))   # move x_vtk_mesh into xb_wall
yb_wall  = np.reshape(y_vtk_mesh , (np.size(y_vtk_mesh [:]),1))   # move y_vtk_mesh into xb_wall

print ('preparing the wall boundry coordinates ends and the number of points was:{}'.format(n_pointsw))


Loading the mesh file and make x & y coordinates ready in variables x & y
preparing the mesh coordinates ends and the number of points was:39238


Loading the inlet Boundry file and make the inlet boundry coordinates ready in variables xb_in & yb_in
preparing the inlet boundry coordinates ends and the number of points was:131


Loading the wall Boundry file and make the wall boundry coordinates ready in variables xb_wall & yb_wall
preparing the wall boundry coordinates ends and the number of points was:760


## Read the 5 sensors Data

In [6]:
#!!specify pts location here:         (sensor locations)
x_data = [1., 1.2, 1.22, 1.31, 1.39 ] 
y_data =[0.15, 0.07, 0.22, 0.036, 0.26 ]
z_data  = [0.,0.,0.,0.,0. ]

x_data = np.asarray(x_data)  #convert to numpy 
y_data = np.asarray(y_data) #convert to numpy 

print ('Loading the file data (ground truth) to assign to our 5 sensors in variables data_vel_u (u direction) & data_vel_v(v direction)')
reader = vtk.vtkXMLUnstructuredGridReader()
reader.SetFileName(File_data)
reader.Update()
data_vtk = reader.GetOutput()
n_points = data_vtk.GetNumberOfPoints()
print ('\nthe number of ground truth data in the file data was:' ,n_points)

VTKpoints = vtk.vtkPoints()
for i in range(len(x_data)): 
	VTKpoints.InsertPoint(i, x_data[i] , y_data[i]  , z_data[i])
point_data = vtk.vtkUnstructuredGrid()
point_data.SetPoints(VTKpoints)
probe = vtk.vtkProbeFilter()
probe.SetInputData(point_data)
probe.SetSourceData(data_vtk)
probe.Update()
array = probe.GetOutput().GetPointData().GetArray(fieldname)
data_vel = VN.vtk_to_numpy(array)  # the value of u and v direction in the 5 sensor locations

data_vel_u = data_vel[:,0]
data_vel_v = data_vel[:,1]


X_scale = 2.0 #The length of the  domain for locating the sensor was 2 now we have to divide the x coordinates by 2 to be like the actual mesh 
              #(need longer length for separation region)
x_data = x_data / X_scale


print('sensor coordinates x:{} y:{} '.format(x_data,y_data))
print('Sensor value in directio of u: ',data_vel_u)
print('Sensor value in directio of v: ',data_vel_v)

# to be confident that these parameters are an array in one column
x_data= x_data.reshape(-1, 1) #need to reshape to get 2D array
y_data= y_data.reshape(-1, 1) #need to reshape to get 2D array
data_vel_u= data_vel_u.reshape(-1, 1) #need to reshape to get 2D array
data_vel_v= data_vel_v.reshape(-1, 1) #need to reshape to get 2D array

Loading the file data (ground truth) to assign to our 5 sensors in variables data_vel_u (u direction) & data_vel_v(v direction)

the number of ground truth data in the file data was: 21140
sensor coordinates x:[0.5   0.6   0.61  0.655 0.695] y:[0.15  0.07  0.22  0.036 0.26 ] 
Sensor value in directio of u:  [0.82861364 0.21744607 0.3266157  0.03768962 0.08355277]
Sensor value in directio of v:  [ 1.0263980e-05 -1.6375961e-02  2.1537246e-02 -8.2011754e-03
  9.3381843e-03]


## Define the wall Boundry conditions, and inlet boundry conditions, then reshape them to be prepared

In [7]:
# defining the boundry conditions
U_BC_in = 0.5
u_in_BC = (yb_in[:]) * ( 0.3 - yb_in[:] )  / 0.0225 * U_BC_in #parabolic  #inlet boundry condition in direction u
v_in_BC = np.linspace(0., 0., n_points)                                   #inlet boundry condition in direction v
u_wall_BC = np.linspace(0., 0., n_pointsw)                                #wall boundry condition in direction u
v_wall_BC = np.linspace(0., 0., n_pointsw)                                #wall boundry condition in direction v

# to be confident that these parameters are an array in one column
xb_wall= xb_wall.reshape(-1, 1) #need to reshape to get 2D array
yb_wall= yb_wall.reshape(-1, 1) #need to reshape to get 2D array
u_wall_BC= u_wall_BC.reshape(-1, 1) #need to reshape to get 2D array
v_wall_BC= v_wall_BC.reshape(-1, 1) #need to reshape to get 2D array
xb_in= xb_in.reshape(-1, 1) #need to reshape to get 2D array
yb_in= yb_in.reshape(-1, 1) #need to reshape to get 2D array
u_in_BC= u_in_BC.reshape(-1, 1) #need to reshape to get 2D array
v_in_BC= v_in_BC.reshape(-1, 1) #need to reshape to get 2D array
print('shape of xb_wall (x coordinates in the wall boundry)',xb_wall.shape)
print('shape of yb_wall (y coordinates in the wall boundry)',yb_wall.shape)
print('shape of u_wall_BC (value of the wall boundry in direction of u)',u_wall_BC.shape)



shape of xb_wall (x coordinates in the wall boundry) (760, 1)
shape of yb_wall (y coordinates in the wall boundry) (760, 1)
shape of u_wall_BC (value of the wall boundry in direction of u) (760, 1)


# Define some properties for the network and training step

In [8]:
Flag_batch = True 
Flag_BC_exact = False 
Flag_pretrain = False # True #If true reads the nets from last run (for train or use pretrained network)
path = "Results/"
batchsize = 256 
epochs  = 100

path = "/content/drive/My Drive/stenosis/Results/"		# saving network

if not os.path.isfile(path+"loss_siren.xlsx"):
	headers= ['Loss_eqn','Loss_BC','Loss_Data','Loss_total','Time']
	workbook_name = path+'loss_siren.xlsx'
	wb = Workbook()
	page = wb.active
	page.title = 'Siren-Based-Stenosis'
	page.append(headers) # write the headers to the first line
	wb.save(filename=workbook_name)

learning_rate = 1e-5 
Flag_schedule = True #If true change the learning rate during training
if (Flag_schedule):
	learning_rate = 5e-4 #starting learning rate
	step_epoch = 500 #1000
	decay_rate = 0.1 # 0.1

# parameteres of the equation
Diff = 0.001
rho = 1.
T = 0.5 #total duraction
#nPt_time = 50 #number of time-steps

Lambda_BC  = 20.
nPt = 130  

path = "/content/drive/My Drive/stenosis/Results/"		# saving network



Flag_x_length = True #if True scales the eqn such that the length of the domain is = X_scale
X_scale = 2.0 #The length of the  domain (need longer length for separation region)
Y_scale = 1.0 
U_scale = 1.0

Lambda_data = 1.

# Train


In [ ]:

net = geo_train(device,x,y,xb_wall,yb_wall,u_wall_BC,v_wall_BC,x_data,y_data,data_vel_u,data_vel_v
          ,batchsize,learning_rate,epochs,path,Flag_batch,Diff,rho,Flag_BC_exact,Lambda_BC,nPt,T,xb_in,yb_in,u_in_BC,v_in_BC )

Reading (pretrain) functions first...
Train Epoch: 0 [0/39238 (0%)]	Loss eqn 0.0013407723 Loss BC 0.00002032 Loss data 0.00005658
Train Epoch: 0 [10240/39238 (26%)]	Loss eqn 0.0021873466 Loss BC 0.00002104 Loss data 0.00021859
Train Epoch: 0 [20480/39238 (52%)]	Loss eqn 0.0013058684 Loss BC 0.00001228 Loss data 0.00002691
Train Epoch: 0 [30720/39238 (78%)]	Loss eqn 0.0024156671 Loss BC 0.00002250 Loss data 0.00003068
*****Total avg Loss : Loss eqn 0.0029763693 Loss BC 0.0000530294 Loss data 0.0001567606 ****
learning rate is  0.0005
Train Epoch: 1 [0/39238 (0%)]	Loss eqn 0.0005642610 Loss BC 0.00001444 Loss data 0.00003347
Train Epoch: 1 [10240/39238 (26%)]	Loss eqn 0.0008453216 Loss BC 0.00001754 Loss data 0.00006015
Train Epoch: 1 [20480/39238 (52%)]	Loss eqn 0.0051190145 Loss BC 0.00002890 Loss data 0.00015544
Train Epoch: 1 [30720/39238 (78%)]	Loss eqn 0.0017030055 Loss BC 0.00002019 Loss data 0.00002493
*****Total avg Loss : Loss eqn 0.0017017551 Loss BC 0.0000235869 Loss data 0.0

# Just run this part if you want to check the outputs after the training

In [ ]:
x_result = torch.Tensor(x).to(device)
y_result = torch.Tensor(y).to(device)
net_in = torch.cat((x_result.requires_grad_(),y_result.requires_grad_()),1)
outneti = net(net_in)  #evaluate model (runs out of memory for large GPU problems!)
output_u = outneti[:,0]
output_v = outneti[:,1]  #evaluate model
output_p = outneti[:,2] 
output_u = output_u.cpu().data.numpy() #need to convert to cpu before converting to numpy
output_v = output_v.cpu().data.numpy()
output_p = output_p.cpu().data.numpy()
npoints = len(x)
z = np.zeros((npoints,1))
u = output_u
v = output_v
vz = np.zeros((len(v),1))
p = output_p
v=np.float64(v)
u=np.float64(u)
pointsToVTK(path+"./output_siren", x, y, z, data = {"velocity" : (np.array(u),np.array(v),vz), "pressure" : p})